In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
import pickle as pk
import csv
import datetime
import lightgbm as lgb
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

#from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import accuracy_score
import re
import time

In [177]:
train_X_tr = pd.read_csv('D:/transaction_risk_competition/transaction_train_new.csv')

In [6]:
train_X_tr.head(10)

,UID,channel,day,time,trans_amt,amt_src1,merchant,code1,code2,trans_type1,...,ip1,bal,amt_src2,acc_id2,acc_id3,geo_code,trans_type2,market_code,market_type,ip1_sub
0,19092,102,30,11:48:47,1459,acdbdb842ac20f1e,abc6bc660561e0eb,NaN,NaN,26bcf43a19df14c8,...,a664530c1fb20abe,1007,9fefed0a981dcb7a,NaN,NaN,wsbq,102.0,2ecf94369847c748,1.0,45f4ca1f4e9b9cb7
1,13465,140,23,16:58:03,725,4d7831c6f695ab19,7a87b1aa20972d48,NaN,NaN,c2f2023d279665b2,...,52abe1e300a8aa10,100,NaN,NaN,NaN,wte7,105.0,NaN,NaN,2b416f1d7e89ad3b
2,13713,140,22,09:56:39,127,c5fc631370cabc0d,cfd035fc7e0a53b2,NaN,NaN,c2f2023d279665b2,...,e4c132d02ff6c095,100,NaN,NaN,NaN,ws7n,105.0,NaN,NaN,ff71a556d86c4b8f
3,22703,140,1,18:11:45,99317,a571c7fda8b7df37,c71c876b8979028e,NaN,NaN,c2f2023d279665b2,...,e1c78a56cc9dfc8b,100,fbf6bf3c8927414c,NaN,NaN,wqj6,NaN,NaN,NaN,cf1527a0b4173e8f
4,17816,140,22,13:04:02,3361,c5fc631370cabc0d,6628f5ca013e3582,NaN,NaN,c2f2023d279665b2,...,abb6b4b8f15be6d6,100,NaN,NaN,NaN,NaN,105.0,NaN,NaN,cbc62c2b1cb7deb9
5,12037,102,8,01:09:29,5536,d7de70fc65292e41,3f6d3d0f42519ea4,NaN,NaN,6d55c54c8b1056fb,...,35d6ccb3b4ff776a,100,079946f395cc0cf1,NaN,NaN,wwhu,102.0,NaN,NaN,1488e4c1cc8f1c55
6,18983,140,10,10:31:27,135470,27c42480134c0d02,7c805cb2868bb52b,NaN,NaN,c2f2023d279665b2,...,26958916ef8a31c4,100,c82bae1f6ebe5674,NaN,NaN,wtbx,105.0,NaN,NaN,597d295ddce706f7
7,12741,140,23,11:56:18,1459,f29829bc82459191,d11d91b96fe7a2ef,NaN,NaN,c2f2023d279665b2,...,01cb473697bc732e,100,9a8ee16bde15e38a,NaN,NaN,wtkn,105.0,NaN,NaN,9d3961d2ea1944fc
8,15986,102,1,06:53:27,24020,155c9e1c32bd0fa2,3bd5cf7c40962299,NaN,NaN,61bfb66c928f36ac,...,b2c59b4bfd512cd1,100,cf6e3a074407c379,01092023fb4f373d,99b877d5d88e8478,wtq6,102.0,NaN,NaN,80e715671282e957
9,10057,102,26,14:14:13,8798,155c9e1c32bd0fa2,d8babe2d19fa0c08,NaN,NaN,6d55c54c8b1056fb,...,aa0a0b28ce6c1757,192,9fefed0a981dcb7a,NaN,NaN,wxrv,102.0,NaN,NaN,3fa05a459ddaa70d


In [7]:
train_X_op = pd.read_csv('D:/transaction_risk_competition/operation_train_new.csv')

In [8]:
train_X_op.head(10)

,UID,day,mode,success,time,os,version,device1,device2,device_code1,device_code2,device_code3,mac1,mac2,ip1,ip2,wifi,geo_code,ip1_sub,ip2_sub
0,10035,30,c8741ce15ceac2a4,1.0,17:51:50,102,7.0.9,49dd36968dbfadda,OPPO R11,ecb58082e0e9b8e2,00fa2e4ba95605ef,NaN,NaN,a8dc52f65085212e,55dd8936655c86f6,NaN,NaN,wskx,e58e48fb9215116e,NaN
1,16264,16,20a91b45ef8f8221,1.0,08:36:00,200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fc7fc47d6c93f554,NaN,NaN,NaN,3502c553ea2ac187
2,13162,8,b668e42707ee9c7b,0.0,18:09:57,102,7.0.5,630a1adff2a87007,MI MAX 2,1da225cb679a37eb,f18538156b8f9b57,NaN,a8dc52f65085212e,NaN,2147d925e7a8ba3c,NaN,NaN,NaN,3591678eca3f7a23,NaN
3,21392,23,c8741ce15ceac2a4,1.0,23:53:49,102,7.0.9,630a1adff2a87007,MI 5X,d0c285513b785609,a7e79e3c2c85c8c7,NaN,NaN,04a2d6c090518faa,c52efc665f0af581,NaN,4ff1553b1c96360e,wm4v,fdb5e20f6a6f2cd7,NaN
4,18599,26,acfaded7e04e7ba0,1.0,11:11:15,102,7.0.0,8399be6b80847240,NaN,27efd832595503f6,b6457e2264266257,NaN,NaN,1ab9dc3eae102aa6,0fe293bea342665a,NaN,NaN,NaN,f5009e2c6c13c80c,NaN
5,20499,27,c8741ce15ceac2a4,1.0,16:44:31,102,7.0.9,a92470e4f878a3f4,M6 NOTE,2bd7dd548de5c1eb,a3f306f70f99d3c5,NaN,NaN,a8dc52f65085212e,907bb247d8ab8fdd,NaN,NaN,ws90,46649b2cd10a7d41,NaN
6,21634,27,c8741ce15ceac2a4,1.0,21:02:12,102,7.0.9,dcc26c88f36170cc,SM-G955F,c50faf7eba139d67,483ddc3a31666b8f,NaN,NaN,b910ea1cd2e10a1d,211d0e54f3da2e8d,NaN,6ec718bbccbaedd2,ws45,d7833f8214e8019f,NaN
7,11305,8,c8741ce15ceac2a4,1.0,18:59:56,102,7.0.5,a92470e4f878a3f4,M3 MAX,39c97da95ebca34e,476e25c0b0bcfa97,NaN,NaN,a8dc52f65085212e,1c682213d73d8f41,NaN,NaN,ws2x,5ec2ddaaa2c6a044,NaN
8,15270,19,479617cd802ed152,1.0,09:12:26,200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa3ae43e9f767109,NaN,NaN,NaN,f9c9ec38dd0f9149
9,10655,16,c8741ce15ceac2a4,1.0,15:25:32,102,7.0.5,8f970961c66d9ec6,ZTE V0840,9778c2c441240765,e7a501e84cf8c9b9,NaN,NaN,a8dc52f65085212e,9549680e109c5d8c,NaN,NaN,ws82,cadff7d686628ff8,NaN


In [10]:
list(train_X_tr.columns)

['UID',
 'channel',
 'day',
 'time',
 'trans_amt',
 'amt_src1',
 'merchant',
 'code1',
 'code2',
 'trans_type1',
 'acc_id1',
 'device_code1',
 'device_code2',
 'device_code3',
 'device1',
 'device2',
 'mac1',
 'ip1',
 'bal',
 'amt_src2',
 'acc_id2',
 'acc_id3',
 'geo_code',
 'trans_type2',
 'market_code',
 'market_type',
 'ip1_sub']

In [178]:
train_X_tr=train_X_tr[['UID','day','time','trans_amt','bal','acc_id1','acc_id2','acc_id3','amt_src1','amt_src2','trans_type2','trans_type1','market_code','market_type','merchant','code1','code2','channel','device_code1','device_code2','device_code3','device1','device2','mac1','ip1','ip1_sub','geo_code']]

In [92]:
train_X_tr.head(6)

,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type2,trans_type1,market_code,market_type,merchant,code1,code2,channel,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub,geo_code
0,19092,30,11:48:47,1459,1007,544dfdeedad0ae67,NaN,NaN,acdbdb842ac20f1e,9fefed0a981dcb7a,102.0,26bcf43a19df14c8,2ecf94369847c748,1.0,abc6bc660561e0eb,NaN,NaN,102,ffda8eaa413171cb,b36097d8a79e5b43,NaN,d2cf44cec09806cc,VIVO X9L,7aaec0f23bfa60ea,a664530c1fb20abe,45f4ca1f4e9b9cb7,wsbq
1,13465,23,16:58:03,725,100,NaN,NaN,NaN,4d7831c6f695ab19,NaN,105.0,c2f2023d279665b2,NaN,NaN,7a87b1aa20972d48,NaN,NaN,140,0e305dafdd3af261,365ec16ad4006c4f,NaN,09baf2f39bc3dc86,NEM-AL10,6f675a9b2124d2f8,52abe1e300a8aa10,2b416f1d7e89ad3b,wte7
2,13713,22,09:56:39,127,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,NaN,cfd035fc7e0a53b2,NaN,NaN,140,429d5656ba8f3d01,760cfbfc0c2e3b49,NaN,a92470e4f878a3f4,M3X,99a5f1ff7f3e54f0,e4c132d02ff6c095,ff71a556d86c4b8f,ws7n
3,22703,1,18:11:45,99317,100,516e596377b2019c,NaN,NaN,a571c7fda8b7df37,fbf6bf3c8927414c,NaN,c2f2023d279665b2,NaN,NaN,c71c876b8979028e,NaN,NaN,140,609954adce541306,31989e264685a18c,NaN,09baf2f39bc3dc86,KNT-UL10,a8dc52f65085212e,e1c78a56cc9dfc8b,cf1527a0b4173e8f,wqj6
4,17816,22,13:04:02,3361,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,NaN,6628f5ca013e3582,NaN,NaN,140,NaN,NaN,2f61e8f52151bd8b,aca4977fbe8741e1,IPHONE 5,NaN,abb6b4b8f15be6d6,cbc62c2b1cb7deb9,NaN
5,12037,8,01:09:29,5536,100,NaN,NaN,NaN,d7de70fc65292e41,079946f395cc0cf1,102.0,6d55c54c8b1056fb,NaN,NaN,3f6d3d0f42519ea4,NaN,NaN,102,NaN,NaN,6113d5f9c8a65ea8,aca4977fbe8741e1,IPHONE 5S,NaN,35d6ccb3b4ff776a,1488e4c1cc8f1c55,wwhu


In [179]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
#pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

train_X_tr.head(1)

,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type2,trans_type1,market_code,market_type,merchant,code1,code2,channel,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub,geo_code
0,19092,30,11:48:47,1459,1007,544dfdeedad0ae67,NaN,NaN,acdbdb842ac20f1e,9fefed0a981dcb7a,102.0,26bcf43a19df14c8,2ecf94369847c748,1.0,abc6bc660561e0eb,NaN,NaN,102,ffda8eaa413171cb,b36097d8a79e5b43,NaN,d2cf44cec09806cc,VIVO X9L,7aaec0f23bfa60ea,a664530c1fb20abe,45f4ca1f4e9b9cb7,wsbq


In [94]:
#统计各列字段的不完整性
#train_X_tr['acc_id1'].value_counts()#只计算非空值
#train_X_tr['acc_id1'].apply(lambda x :x/df['Prices'].sum())
#train_X_tr['acc_id1'].count()

# len_count=len(train_X_tr['UID'])
# for col in ['UID','day','time','trans_amt','bal','acc_id1','acc_id2','acc_id3','amt_src1','amt_src2','trans_type2','trans_type1','market_code','market_type','merchant','code1','code2','channel','device_code1','device_code2','device_code3','device1','device2','mac1','ip1','ip1_sub','geo_code']:
#     print('{0}'.format(col),  '%.2f' %(train_X_tr[col].isnull().sum()/len_count))

#需要因子化的列，
#train_X_tr['amt_src1'].value_counts()
#显示所有行
pd.set_option('display.max_rows', None)
train_X_tr['geo_code'].value_counts()



wt7j    7312
wqq2    2518
wxkz    1994
ws06    1889
wxsb    1769
wmgu    1708
wtfh    1559
webw    1506
wkp7    1469
ws3j    1468
ws0e    1379
wt5c    1363
w7w3    1303
ww9w    1209
wsbq    1130
ww5j    1129
ws2r    1101
wmd6    1083
wmr8    1016
wkzc     964
wwj1     957
weby     921
wmuh     915
wt59     852
wm65     825
wtsk     791
ws07     789
wme1     788
wqnr     781
wth5     752
ww8y     740
ww4g     736
wxy3     726
y8th     715
ws4u     703
ws05     702
wm4u     687
wtq3     685
wrhr     677
vb79     653
wmd0     648
wt58     646
wt5f     635
wt5t     628
wss8     620
ww9x     615
wqj7     607
ws7n     580
wmu5     576
wsb1     572
ws3n     570
wxrt     565
ws1x     565
ws2y     558
wm4x     556
wwc2     552
wx85     552
wmeb     550
wtg1     538
wxs8     534
wth4     520
y8tp     516
wxss     516
ws3b     510
wts4     509
wkxw     506
wxrv     506
wm57     484
ws4t     474
ww8z     472
wxy2     468
wtem     458
wt02     455
ws2n     443
tzy4     443
wx4g     431
wkj1     425

In [180]:
#删除地理信息，目前不会挖掘
train_X_tr.drop('geo_code',axis = 1,inplace = True)
#train_X_tr.drop('acc_id1_isnull',axis = 1,inplace = True)
train_X_tr.head(5)


,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type2,trans_type1,market_code,market_type,merchant,code1,code2,channel,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub
0,19092,30,11:48:47,1459,1007,544dfdeedad0ae67,NaN,NaN,acdbdb842ac20f1e,9fefed0a981dcb7a,102.0,26bcf43a19df14c8,2ecf94369847c748,1.0,abc6bc660561e0eb,NaN,NaN,102,ffda8eaa413171cb,b36097d8a79e5b43,NaN,d2cf44cec09806cc,VIVO X9L,7aaec0f23bfa60ea,a664530c1fb20abe,45f4ca1f4e9b9cb7
1,13465,23,16:58:03,725,100,NaN,NaN,NaN,4d7831c6f695ab19,NaN,105.0,c2f2023d279665b2,NaN,NaN,7a87b1aa20972d48,NaN,NaN,140,0e305dafdd3af261,365ec16ad4006c4f,NaN,09baf2f39bc3dc86,NEM-AL10,6f675a9b2124d2f8,52abe1e300a8aa10,2b416f1d7e89ad3b
2,13713,22,09:56:39,127,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,NaN,cfd035fc7e0a53b2,NaN,NaN,140,429d5656ba8f3d01,760cfbfc0c2e3b49,NaN,a92470e4f878a3f4,M3X,99a5f1ff7f3e54f0,e4c132d02ff6c095,ff71a556d86c4b8f
3,22703,1,18:11:45,99317,100,516e596377b2019c,NaN,NaN,a571c7fda8b7df37,fbf6bf3c8927414c,NaN,c2f2023d279665b2,NaN,NaN,c71c876b8979028e,NaN,NaN,140,609954adce541306,31989e264685a18c,NaN,09baf2f39bc3dc86,KNT-UL10,a8dc52f65085212e,e1c78a56cc9dfc8b,cf1527a0b4173e8f
4,17816,22,13:04:02,3361,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,NaN,6628f5ca013e3582,NaN,NaN,140,NaN,NaN,2f61e8f52151bd8b,aca4977fbe8741e1,IPHONE 5,NaN,abb6b4b8f15be6d6,cbc62c2b1cb7deb9


In [181]:
#选取做因子化
##amt_src1 交易类型，余额，花呗做因子化
##trans_type2	线上线下 ，一定要因子化，只有4中，加上none,5中
##market_type	，营销类型，只有2种，加上空值，三种，一定要因子化
##channel 平台，渠道，5种，加上空值6种，适合因子化
values={'market_type':0}
train_X_tr.fillna(value=values,inplace = True)
train_X_tr.head(5)


,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type2,trans_type1,market_code,market_type,merchant,code1,code2,channel,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub
0,19092,30,11:48:47,1459,1007,544dfdeedad0ae67,NaN,NaN,acdbdb842ac20f1e,9fefed0a981dcb7a,102.0,26bcf43a19df14c8,2ecf94369847c748,1.0,abc6bc660561e0eb,NaN,NaN,102,ffda8eaa413171cb,b36097d8a79e5b43,NaN,d2cf44cec09806cc,VIVO X9L,7aaec0f23bfa60ea,a664530c1fb20abe,45f4ca1f4e9b9cb7
1,13465,23,16:58:03,725,100,NaN,NaN,NaN,4d7831c6f695ab19,NaN,105.0,c2f2023d279665b2,NaN,0.0,7a87b1aa20972d48,NaN,NaN,140,0e305dafdd3af261,365ec16ad4006c4f,NaN,09baf2f39bc3dc86,NEM-AL10,6f675a9b2124d2f8,52abe1e300a8aa10,2b416f1d7e89ad3b
2,13713,22,09:56:39,127,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,0.0,cfd035fc7e0a53b2,NaN,NaN,140,429d5656ba8f3d01,760cfbfc0c2e3b49,NaN,a92470e4f878a3f4,M3X,99a5f1ff7f3e54f0,e4c132d02ff6c095,ff71a556d86c4b8f
3,22703,1,18:11:45,99317,100,516e596377b2019c,NaN,NaN,a571c7fda8b7df37,fbf6bf3c8927414c,NaN,c2f2023d279665b2,NaN,0.0,c71c876b8979028e,NaN,NaN,140,609954adce541306,31989e264685a18c,NaN,09baf2f39bc3dc86,KNT-UL10,a8dc52f65085212e,e1c78a56cc9dfc8b,cf1527a0b4173e8f
4,17816,22,13:04:02,3361,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,0.0,6628f5ca013e3582,NaN,NaN,140,NaN,NaN,2f61e8f52151bd8b,aca4977fbe8741e1,IPHONE 5,NaN,abb6b4b8f15be6d6,cbc62c2b1cb7deb9


In [182]:
#对market_type因子化
train_X_tr['market_type'], uniques =pd.factorize(train_X_tr['market_type'],sort=True)
uniques
del uniques

In [183]:
train_X_tr.head(5)
columns=list(train_X_tr.columns)
columns.remove('market_type')
columns_del_market_type=columns
train_X_tr=pd.get_dummies(train_X_tr['market_type'],prefix='market_type').join(train_X_tr[columns_del_market_type])
train_X_tr.head(5)

,market_type_0,market_type_1,market_type_2,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type2,trans_type1,market_code,merchant,code1,code2,channel,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub
0,0,1,0,19092,30,11:48:47,1459,1007,544dfdeedad0ae67,NaN,NaN,acdbdb842ac20f1e,9fefed0a981dcb7a,102.0,26bcf43a19df14c8,2ecf94369847c748,abc6bc660561e0eb,NaN,NaN,102,ffda8eaa413171cb,b36097d8a79e5b43,NaN,d2cf44cec09806cc,VIVO X9L,7aaec0f23bfa60ea,a664530c1fb20abe,45f4ca1f4e9b9cb7
1,1,0,0,13465,23,16:58:03,725,100,NaN,NaN,NaN,4d7831c6f695ab19,NaN,105.0,c2f2023d279665b2,NaN,7a87b1aa20972d48,NaN,NaN,140,0e305dafdd3af261,365ec16ad4006c4f,NaN,09baf2f39bc3dc86,NEM-AL10,6f675a9b2124d2f8,52abe1e300a8aa10,2b416f1d7e89ad3b
2,1,0,0,13713,22,09:56:39,127,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,cfd035fc7e0a53b2,NaN,NaN,140,429d5656ba8f3d01,760cfbfc0c2e3b49,NaN,a92470e4f878a3f4,M3X,99a5f1ff7f3e54f0,e4c132d02ff6c095,ff71a556d86c4b8f
3,1,0,0,22703,1,18:11:45,99317,100,516e596377b2019c,NaN,NaN,a571c7fda8b7df37,fbf6bf3c8927414c,NaN,c2f2023d279665b2,NaN,c71c876b8979028e,NaN,NaN,140,609954adce541306,31989e264685a18c,NaN,09baf2f39bc3dc86,KNT-UL10,a8dc52f65085212e,e1c78a56cc9dfc8b,cf1527a0b4173e8f
4,1,0,0,17816,22,13:04:02,3361,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,6628f5ca013e3582,NaN,NaN,140,NaN,NaN,2f61e8f52151bd8b,aca4977fbe8741e1,IPHONE 5,NaN,abb6b4b8f15be6d6,cbc62c2b1cb7deb9


In [184]:
values={'channel':0}
train_X_tr.fillna(value=values,inplace = True)
train_X_tr.head(5)

columns=list(train_X_tr.columns)
columns.remove('channel')
columns_del_channel=columns
train_X_tr=pd.get_dummies(train_X_tr['channel'],prefix='channel').join(train_X_tr[columns_del_channel])
train_X_tr.head(5)


,channel_102,channel_106,channel_118,channel_119,channel_140,market_type_0,market_type_1,market_type_2,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type2,trans_type1,market_code,merchant,code1,code2,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub
0,1,0,0,0,0,0,1,0,19092,30,11:48:47,1459,1007,544dfdeedad0ae67,NaN,NaN,acdbdb842ac20f1e,9fefed0a981dcb7a,102.0,26bcf43a19df14c8,2ecf94369847c748,abc6bc660561e0eb,NaN,NaN,ffda8eaa413171cb,b36097d8a79e5b43,NaN,d2cf44cec09806cc,VIVO X9L,7aaec0f23bfa60ea,a664530c1fb20abe,45f4ca1f4e9b9cb7
1,0,0,0,0,1,1,0,0,13465,23,16:58:03,725,100,NaN,NaN,NaN,4d7831c6f695ab19,NaN,105.0,c2f2023d279665b2,NaN,7a87b1aa20972d48,NaN,NaN,0e305dafdd3af261,365ec16ad4006c4f,NaN,09baf2f39bc3dc86,NEM-AL10,6f675a9b2124d2f8,52abe1e300a8aa10,2b416f1d7e89ad3b
2,0,0,0,0,1,1,0,0,13713,22,09:56:39,127,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,cfd035fc7e0a53b2,NaN,NaN,429d5656ba8f3d01,760cfbfc0c2e3b49,NaN,a92470e4f878a3f4,M3X,99a5f1ff7f3e54f0,e4c132d02ff6c095,ff71a556d86c4b8f
3,0,0,0,0,1,1,0,0,22703,1,18:11:45,99317,100,516e596377b2019c,NaN,NaN,a571c7fda8b7df37,fbf6bf3c8927414c,NaN,c2f2023d279665b2,NaN,c71c876b8979028e,NaN,NaN,609954adce541306,31989e264685a18c,NaN,09baf2f39bc3dc86,KNT-UL10,a8dc52f65085212e,e1c78a56cc9dfc8b,cf1527a0b4173e8f
4,0,0,0,0,1,1,0,0,17816,22,13:04:02,3361,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,105.0,c2f2023d279665b2,NaN,6628f5ca013e3582,NaN,NaN,NaN,NaN,2f61e8f52151bd8b,aca4977fbe8741e1,IPHONE 5,NaN,abb6b4b8f15be6d6,cbc62c2b1cb7deb9


In [185]:
values={'trans_type2':0}
train_X_tr.fillna(value=values,inplace = True)
train_X_tr.head(5)

columns=list(train_X_tr.columns)
columns.remove('trans_type2')
columns_del_trans_type2=columns
train_X_tr=pd.get_dummies(train_X_tr['trans_type2'],prefix='trans_type2').join(train_X_tr[columns_del_trans_type2])
train_X_tr.head(5)

,trans_type2_0.0,trans_type2_102.0,trans_type2_103.0,trans_type2_104.0,trans_type2_105.0,channel_102,channel_106,channel_118,channel_119,channel_140,market_type_0,market_type_1,market_type_2,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type1,market_code,merchant,code1,code2,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub
0,0,1,0,0,0,1,0,0,0,0,0,1,0,19092,30,11:48:47,1459,1007,544dfdeedad0ae67,NaN,NaN,acdbdb842ac20f1e,9fefed0a981dcb7a,26bcf43a19df14c8,2ecf94369847c748,abc6bc660561e0eb,NaN,NaN,ffda8eaa413171cb,b36097d8a79e5b43,NaN,d2cf44cec09806cc,VIVO X9L,7aaec0f23bfa60ea,a664530c1fb20abe,45f4ca1f4e9b9cb7
1,0,0,0,0,1,0,0,0,0,1,1,0,0,13465,23,16:58:03,725,100,NaN,NaN,NaN,4d7831c6f695ab19,NaN,c2f2023d279665b2,NaN,7a87b1aa20972d48,NaN,NaN,0e305dafdd3af261,365ec16ad4006c4f,NaN,09baf2f39bc3dc86,NEM-AL10,6f675a9b2124d2f8,52abe1e300a8aa10,2b416f1d7e89ad3b
2,0,0,0,0,1,0,0,0,0,1,1,0,0,13713,22,09:56:39,127,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,c2f2023d279665b2,NaN,cfd035fc7e0a53b2,NaN,NaN,429d5656ba8f3d01,760cfbfc0c2e3b49,NaN,a92470e4f878a3f4,M3X,99a5f1ff7f3e54f0,e4c132d02ff6c095,ff71a556d86c4b8f
3,1,0,0,0,0,0,0,0,0,1,1,0,0,22703,1,18:11:45,99317,100,516e596377b2019c,NaN,NaN,a571c7fda8b7df37,fbf6bf3c8927414c,c2f2023d279665b2,NaN,c71c876b8979028e,NaN,NaN,609954adce541306,31989e264685a18c,NaN,09baf2f39bc3dc86,KNT-UL10,a8dc52f65085212e,e1c78a56cc9dfc8b,cf1527a0b4173e8f
4,0,0,0,0,1,0,0,0,0,1,1,0,0,17816,22,13:04:02,3361,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,c2f2023d279665b2,NaN,6628f5ca013e3582,NaN,NaN,NaN,NaN,2f61e8f52151bd8b,aca4977fbe8741e1,IPHONE 5,NaN,abb6b4b8f15be6d6,cbc62c2b1cb7deb9


In [186]:
values={'trans_type1':'0'}
train_X_tr.fillna(value=values,inplace = True)
train_X_tr.head(5)

train_X_tr['trans_type1'], uniques =pd.factorize(train_X_tr['trans_type1'],sort=True)
uniques
#del uniques

columns=list(train_X_tr.columns)
#columns.remove('trans_type1')
#columns_del_trans_type1=columns
#最好还保留因子化后的trans_type1，放在后面的groupby字段里，同时get_dummies(train_X_tr['trans_type1']得到这笔交易是什么类型
train_X_tr=pd.get_dummies(train_X_tr['trans_type1'],prefix='trans_type1').join(train_X_tr[columns])
train_X_tr.head(5)

,trans_type1_0,trans_type1_1,trans_type1_2,trans_type1_3,trans_type1_4,trans_type1_5,trans_type1_6,trans_type1_7,trans_type1_8,trans_type1_9,trans_type1_10,trans_type1_11,trans_type1_12,trans_type1_13,trans_type1_14,trans_type2_0.0,trans_type2_102.0,trans_type2_103.0,trans_type2_104.0,trans_type2_105.0,channel_102,channel_106,channel_118,channel_119,channel_140,market_type_0,market_type_1,market_type_2,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type1,market_code,merchant,code1,code2,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,19092,30,11:48:47,1459,1007,544dfdeedad0ae67,NaN,NaN,acdbdb842ac20f1e,9fefed0a981dcb7a,0,2ecf94369847c748,abc6bc660561e0eb,NaN,NaN,ffda8eaa413171cb,b36097d8a79e5b43,NaN,d2cf44cec09806cc,VIVO X9L,7aaec0f23bfa60ea,a664530c1fb20abe,45f4ca1f4e9b9cb7
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,13465,23,16:58:03,725,100,NaN,NaN,NaN,4d7831c6f695ab19,NaN,8,NaN,7a87b1aa20972d48,NaN,NaN,0e305dafdd3af261,365ec16ad4006c4f,NaN,09baf2f39bc3dc86,NEM-AL10,6f675a9b2124d2f8,52abe1e300a8aa10,2b416f1d7e89ad3b
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,13713,22,09:56:39,127,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,8,NaN,cfd035fc7e0a53b2,NaN,NaN,429d5656ba8f3d01,760cfbfc0c2e3b49,NaN,a92470e4f878a3f4,M3X,99a5f1ff7f3e54f0,e4c132d02ff6c095,ff71a556d86c4b8f
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,22703,1,18:11:45,99317,100,516e596377b2019c,NaN,NaN,a571c7fda8b7df37,fbf6bf3c8927414c,8,NaN,c71c876b8979028e,NaN,NaN,609954adce541306,31989e264685a18c,NaN,09baf2f39bc3dc86,KNT-UL10,a8dc52f65085212e,e1c78a56cc9dfc8b,cf1527a0b4173e8f
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,17816,22,13:04:02,3361,100,NaN,NaN,NaN,c5fc631370cabc0d,NaN,8,NaN,6628f5ca013e3582,NaN,NaN,NaN,NaN,2f61e8f52151bd8b,aca4977fbe8741e1,IPHONE 5,NaN,abb6b4b8f15be6d6,cbc62c2b1cb7deb9


In [187]:
values={'amt_src1':'0'}
train_X_tr.fillna(value=values,inplace = True)
train_X_tr.head(5)

train_X_tr['amt_src1'], uniques =pd.factorize(train_X_tr['amt_src1'],sort=True)
uniques
#del uniques

columns=list(train_X_tr.columns)
#columns.remove('amt_src1')
#columns_del_amt_src1=columns
#最好还保留因子化后的trans_type1，放在后面的groupby字段里，同时get_dummies(train_X_tr['amt_src1']得到这笔交易是什么类型
train_X_tr=pd.get_dummies(train_X_tr['amt_src1'],prefix='amt_src1').join(train_X_tr[columns])
train_X_tr.head(5)

,amt_src1_0,amt_src1_1,amt_src1_2,amt_src1_3,amt_src1_4,amt_src1_5,amt_src1_6,amt_src1_7,amt_src1_8,amt_src1_9,amt_src1_10,amt_src1_11,amt_src1_12,amt_src1_13,amt_src1_14,amt_src1_15,amt_src1_16,amt_src1_17,amt_src1_18,amt_src1_19,amt_src1_20,amt_src1_21,amt_src1_22,amt_src1_23,amt_src1_24,amt_src1_25,amt_src1_26,amt_src1_27,trans_type1_0,trans_type1_1,trans_type1_2,trans_type1_3,trans_type1_4,trans_type1_5,trans_type1_6,trans_type1_7,trans_type1_8,trans_type1_9,trans_type1_10,trans_type1_11,trans_type1_12,trans_type1_13,trans_type1_14,trans_type2_0.0,trans_type2_102.0,trans_type2_103.0,trans_type2_104.0,trans_type2_105.0,channel_102,channel_106,channel_118,channel_119,channel_140,market_type_0,market_type_1,market_type_2,UID,day,time,trans_amt,bal,acc_id1,acc_id2,acc_id3,amt_src1,amt_src2,trans_type1,market_code,merchant,code1,code2,device_code1,device_code2,device_code3,device1,device2,mac1,ip1,ip1_sub
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,19092,30,11:48:47,1459,1007,544dfdeedad0ae67,NaN,NaN,18,9fefed0a981dcb7a,0,2ecf94369847c748,abc6bc660561e0eb,NaN,NaN,ffda8eaa413171cb,b36097d8a79e5b43,NaN,d2cf44cec09806cc,VIVO X9L,7aaec0f23bfa60ea,a664530c1fb20abe,45f4ca1f4e9b9cb7
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,13465,23,16:58:03,725,100,NaN,NaN,NaN,7,NaN,8,NaN,7a87b1aa20972d48,NaN,NaN,0e305dafdd3af261,365ec16ad4006c4f,NaN,09baf2f39bc3dc86,NEM-AL10,6f675a9b2124d2f8,52abe1e300a8aa10,2b416f1d7e89ad3b
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,13713,22,09:56:39,127,100,NaN,NaN,NaN,21,NaN,8,NaN,cfd035fc7e0a53b2,NaN,NaN,429d5656ba8f3d01,760cfbfc0c2e3b49,NaN,a92470e4f878a3f4,M3X,99a5f1ff7f3e54f0,e4c132d02ff6c095,ff71a556d86c4b8f
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,22703,1,18:11:45,99317,100,516e596377b2019c,NaN,NaN,17,fbf6bf3c8927414c,8,NaN,c71c876b8979028e,NaN,NaN,609954adce541306,31989e264685a18c,NaN,09baf2f39bc3dc86,KNT-UL10,a8dc52f65085212e,e1c78a56cc9dfc8b,cf1527a0b4173e8f
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,17816,22,13:04:02,3361,100,NaN,NaN,NaN,21,NaN,8,NaN,6628f5ca013e3582,NaN,NaN,NaN,NaN,2f61e8f52151bd8b,aca4977fbe8741e1,IPHONE 5,NaN,abb6b4b8f15be6d6,cbc62c2b1cb7deb9
